In [27]:
import json
import re
import numpy as np
import torch
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data
from sklearn.model_selection import train_test_split
import torch_geometric.transforms as T
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [23]:
test1 = torch.zeros(10, dtype = torch.bool)
docList = ["fish", "bitch", "nitch", "ditch", "witch"]
labels = ["11", "92", "10", "15", "21"]

data = Data(x = docList, y = labels)

m = train_test_split(pd.Series(docList), pd.Series(["11", "92", "10", "15", "21"]), test_size= 0.4)

train_mask = torch.zeros(11, dtype = torch.bool)
train_mask[m[0].index + 6] = True
data['train_mask'] = train_mask


print(data)
print(data['train_mask'])
print(data.y)


Data(x=[5], y=[5], train_mask=[11])
tensor([False, False, False, False, False, False,  True,  True, False, False,
         True])
['11', '92', '10', '15', '21']


In [64]:

# create labels and sentences dataset.
monthsDict = {"Jan": 0, "Feb": 31, "Mar": 59, "Apr": 90, "May": 120, "Jun": 151, "Jul": 181, "Aug": 212, "Sep": 243, "Oct": 273, "Nov": 304, "Dec": 334}
counter = 0
for article in json_data: 
    counter += 1
    # convert dates into floats
    preDate = article['date'].split(" ")
    newDate = int(preDate[2]) + (int(preDate[0]) + monthsDict[preDate[1]]) / 365

    # remove punctuation and redundancies from sentences
    sentence = ""
    for i in article['body']:
        sentence +=" " + i            
    sentence = re.sub(r"[!@#$%^&*.]",'',sentence)
    print(sentence)
    if counter == 5:
        break

 Sir: It was interesting to read Francesca Unsworth boast in her letter about Melanie Phillips article on Alan Johnston, that the BBC 'is reporting from the region fairly and impartially'. If that is the case, why won't the BBC release the Balen report which was commissioned specifically to consider whether that was true? Hyarn Lehrer London NW11
 Q. I am godmother to a dear eight-year-old boy whose parents are separated. Every so often I try to see the little chap by inviting him to lunch in a smart restaurant for a treat. However the last two times that I have done this his father has trumped me by coming too and insisting on paying for the lunch, in fact settling it with the maître d’ behind my back. How can I get around this difficulty, Mary, since the boy’s birthday looms and once again I have arranged to take him to his favourite restaurant? S.L.B., Barnes
 Telling tales Jaspistos IN COMPETITION NO. 2128 you were invited to invent comic tales which could be entitled 'The ivy-tree

In [12]:
print(np.identity(5))

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [41]:
class TextDataset(InMemoryDataset):
    def __init__(self, docList, labels, nodeCount, test_size, transform = None):
        super(TextDataset, self).__init__('.', transform = transform)

        data = Data(x = docList, y = labels)

        # split data into training, validation(?), test
        train_data, test_data, train_labels, test_labels = train_test_split(pd.Series(docList), pd.Series(labels), test_size = test_size)

        self.x_train = train_data
        self.x_test = test_data
        self.y_train = train_labels
        self.y_test = test_labels

        vocabCount = nodeCount - len(docList)
        # create masks
        self.train_mask = torch.zeros(nodeCount, dtype = torch.bool)
        self.test_mask = torch.zeros(nodeCount, dtype = torch.bool)

        self.train_mask[train_data.index + vocabCount] = True
        self.test_mask[test_data.index + vocabCount] = True

        data['train_mask'] = self.train_mask
        data['test_mask'] = self.test_mask
        self.data, self.slices = self.collate([data])

    def _download(self):
        return
    
    def _process(self):
        return
    
    def __repr__(self) -> str:
        return super().__repr__()
    

def get_vocab(docList, docNames):
    """
    This function takes the dataset and generates the list of vocab with their number of appearances as well using CountVectorizer.
    """

    # docList = [] # get_vocab(datapath + "_docs.txt")
    # docNames = []
    # f = open(docPath, 'rb')
    # for line in f.readlines():
    #     docList.append(line.decode('UTF-8').split('\t')[1].strip())
    #     docNames.append(line.decode('UTF-8').split('\t')[0].strip())
    # f.close()


    tfidf = TfidfVectorizer(min_df=3)
    tfidfvect = tfidf.fit_transform(docList)
    vocab = tfidf.get_feature_names_out()
    df_tfidfvect = pd.DataFrame(data = tfidfvect.toarray(),index = docNames, columns = vocab)
    print("Vocab is complete")

    return vocab, tfidfvect, df_tfidfvect





In [44]:
docList = [] # get_vocab(datapath + "_docs.txt")
docNames = []
f = open('data/gutenold/guten_sentences.txt', 'rb')
for line in f.readlines():
    docList.append(line.decode('UTF-8').split('\t')[1].strip())
    docNames.append(line.decode('UTF-8').split('\t')[0].strip())
f.close()
numDocs = len(docList)

wordList, tfvect, df_tfidf = get_vocab(docList, docNames)
numWords = len(wordList)
count = 0
numNodes =  numWords + numDocs

labels = []
f = open('data/gutenold/guten_sentences.txt', 'rb')
for line in f.readlines():
    labels.append(line.decode('UTF-8').split('\t')[1].strip())
f.close()

# create labels and sentences dataset.

data = TextDataset(docList, labels, numNodes, test_size =0.2)

print(type(data.train_mask))


Vocab is complete
<class 'torch.Tensor'>
